# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 72 new papers today
          39 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/38 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.01397


extracting tarball to tmp_2212.01397...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.01397/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'bib' from 'tmp_2212.01397/bib.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2212.02458...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.02491


extracting tarball to tmp_2212.02491...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.02491/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01397-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01397) | **The Star Formation History of the Milky Way's Nuclear Star Cluster**  |
|| Zhuo Chen, et al. -- incl., <mark>Anja Feldmeier-Krause</mark> |
|*Appeared on*| *2022-12-06*|
|*Comments*| *37 pages, 23 figures, Accepted to ApJ*|
|**Abstract**| We report the first star formation history study of the Milky Way's nuclear star cluster (NSC) that includes observational constraints from a large sample of stellar metallicity measurements. These metallicity measurements were obtained from recent surveys from Gemini and VLT of 770 late-type stars within the central 1.5 pc. These metallicity measurements, along with photometry and spectroscopically derived temperatures, are forward modeled with a Bayesian inference approach. Including metallicity measurements improves the overall fit quality, as the low-temperature red giants that were previously difficult to constrain are now accounted for, and the best fit favors a two-component model. The dominant component contains 93%$\pm$3% of the mass, is metal-rich ($\overline{[M/H]}\sim$0.45), and has an age of 5$^{+3}_{-2}$ Gyr, which is $\sim$3 Gyr younger than earlier studies with fixed (solar) metallicity; this younger age challenges co-evolutionary models in which the NSC and supermassive black holes formed simultaneously at early times. The minor population component has low metallicity ($\overline{[M/H]}\sim$ -1.1) and contains $\sim$7% of the stellar mass. The age of the minor component is uncertain (0.1 - 5 Gyr old). Using the estimated parameters, we infer the following NSC stellar remnant population (with $\sim$18% uncertainty): 1.5$\times$10$^5$ neutron stars, 2.5$\times$10$^5$ stellar mass black holes (BHs) and 2.2$\times$10$^4$ BH-BH binaries. These predictions result in 2-4 times fewer neutron stars compared to earlier predictions that assume solar metallicity, introducing a possible new path to understand the so-called "missing pulsar problem". Finally, we present updated predictions for the BH-BH merger rates (0.01-3 Gpc$^{-3}$yr$^{-1}$). |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01526-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01526) | **PHANGS-JWST First Results: The 21 $μ$m Compact Source Population**  |
|| Hamid Hassani, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>Frank Bigiel</mark>, <mark>Kathryn Kreckel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-12-06*|
|*Comments*| *15 pages, 5 figures, Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| We use PHANGS-JWST data to identify and classify 1271 compact 21 $\mu$m sources in four nearby galaxies using MIRI F2100W data. We identify sources using a dendrogram-based algorithm, and we measure the background-subtracted flux densities for JWST bands from 2 $\mu$m to 21 $\mu$m. Using the SED in JWST as well as HST bands, plus ALMA and MUSE/VLT observations, we classify the sources by eye. Then we use this classification to define regions in color-color space, and so establish a quantitative framework for classifying sources. We identify 1085 sources as belonging to the ISM of the target galaxies with the remainder being dusty stars or background galaxies. These 21 $\mu$m sources are strongly spatially associated with HII regions ($>92\%$ of sources), while 74$\%$ of sources are coincident with a stellar association defined in the HST data. Using SED fitting, we find that the stellar masses of the 21 $\mu$m sources span a range of 10$^{2}$ to 10$^{4}~M_\odot$ with mass-weighted ages down to 2 Myr. There is a tight correlation between attenuation-corrected H$\alpha$ and 21 $\mu$m luminosity for $L_{\nu,\mathrm{F2100W}}>10^{19}~\mathrm{W~Hz}^{-1}$. Young embedded source candidates selected at 21 $\mu$m are found below this threshold and have $M_\star < 10^{3}~M_\odot$. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01707-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01707) | **Powerful yet lonely: Is 3C 297 a high-redshift fossil group?**  |
|| Valentina Missaglia, et al. -- incl., <mark>Mischa Schirmer</mark> |
|*Appeared on*| *2022-12-06*|
|*Comments*| *The Astrophysical Journal Supplement Series, in press*|
|**Abstract**| The environment of the high-redshift (z=1.408), powerful radio-loud galaxy 3C 297 has several distinctive features of a galaxy cluster. Among them, a characteristic halo of hot gas revealed by Chandra X-ray observations. In addition, a radio map obtained with the Very Large Array (VLA) shows a bright hotspot in the northwestern direction, created by the interaction of the AGN jet arising from 3C 297 with its environment. In the X-ray images, emission cospatial with the northwestern radio lobe is detected, and peaks at the position of the radio hotspot. The extended, complex X-ray emission observed with our new Chandra data is largely unrelated to its radio structure. Despite having attributes of a galaxy cluster, no companion galaxies have been identified from 39 new spectra of neighboring targets of 3C 297 obtained with the Gemini Multi-Object Spectrograph. None of the 19 galaxies for which a redshift was determined lies at the same distance as 3C 297. The optical spectral analysis of the new Gemini spectrum of 3C 297 reveals an isolated Type-II radio-loud AGN. We also detected line broadening in [O II](3728) with a FWHM about 1700 km/s and possible line shifts of up to 500-600 km/s. We postulate that the host galaxy of 3C 297 is a fossil group, in which most of the stellar mass has merged into a single object, leaving behind an X-ray halo. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02235-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02235) | **CONCERTO: Simulating the CO, [CII], and [CI] line emission of galaxies  in a 117 $\rm deg^2$ field and the impact of field-to-field variance**  |
|| A. Gkogkou, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2022-12-06*|
|*Comments*| **|
|**Abstract**| In the submm regime, spectral line scans and line intensity mapping (LIM) are new promising probes for the cold gas content and star formation rate of galaxies across cosmic time. However, both of these two measurements suffer from field-to-field variance. We study the effect of field-to-field variance on the predicted CO and [CII] power spectra from future LIM experiments such as CONCERTO, as well as on the line luminosity functions (LFs) and the cosmic molecular gas mass density that are currently derived from spectral line scans. We combined a 117 $\rm deg^2$ dark matter lightcone from the Uchuu cosmological simulation with the simulated infrared dusty extragalactic sky (SIDES) approach. We find that in order to constrain the CO LF with an uncertainty below 20%, we need survey sizes of at least 0.1 $\rm deg^2$. Furthermore, accounting for the field-to-field variance using only the Poisson variance can underestimate the total variance by up to 80%. The lower the luminosity is and the larger the survey size is, the higher the level of underestimate. At $z$<3, the impact of field-to-field variance on the cosmic molecular gas density can be as high as 40% for the 4.6 arcmin$^2$ field, but drops below 10% for areas larger than 0.2 deg$^2$. However, at $z>3$ the variance decreases more slowly with survey size and for example drops below 10% for 1 deg$^2$ fields. Finally, we find that the CO and [CII] LIM power spectra can vary by up to 50% in $\rm 1 deg^2$ fields. This limits the accuracy of the constraints provided by the first 1 deg$^2$ surveys. The level of the shot noise power is always dominated by the sources that are just below the detection thresholds. We provide an analytical formula to estimate the field-to-field variance of current or future LIM experiments. The code and the full SIDES-Uchuu products (catalogs, cubes, and maps) are publicly available. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02357-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02357) | **Astrometric precision tests on TESS data**  |
|| Mario Gai, et al. -- incl., <mark>Zhaoxiang Qi</mark> |
|*Appeared on*| *2022-12-06*|
|*Comments*| *13 pages, 8 figures*|
|**Abstract**| Background. Astrometry at or below the micro-arcsec level with an imaging telescope assumes that the uncertainty on the location of an unresolved source can be an arbitrarily small fraction of the detector pixel, given a sufficient photon budget. Aim. This paper investigates the geometric limiting precision, in terms of CCD pixel fraction, achieved by a large set of star field images, selected among the publicly available science data of the TESS mission. Method. The statistics of the distance between selected bright stars ($G \simeq 5\,mag$), in pixel units, is evaluated, using the position estimate provided in the TESS light curve files. Results. The dispersion of coordinate differences appears to be affected by long term variation and noisy periods, at the level of $0.01$ pixel. The residuals with respect to low-pass filtered data (tracing the secular evolution), which are interpreted as the experimental astrometric noise, reach the level of a few milli-pixel or below, down to $1/5,900$ pixel. Saturated images are present, evidencing that the astrometric precision is mostly preserved across the CCD columns, whereas it features a graceful degradation in the along column direction. The cumulative performance of the image set is a few micro-pixel across columns, or a few 10 micro-pixel along columns. Conclusions. The idea of astrometric precision down to a small fraction of a CCD pixel, given sufficient signal to noise ratio, is confirmed by real data from an in-flight science instrument to the $10^{-6}$ pixel level. Implications for future high precision astrometry missions are briefly discussed. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02423-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02423) | **Ultraviolet imaging observations of three jellyfish galaxies: Star  formation suppression in the centre and ongoing star formation in stripped  tails**  |
|| Koshy George, et al. -- incl., <mark>S. K. Ghosh</mark> |
|*Appeared on*| *2022-12-06*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**| Spiral galaxies undergo strong ram-pressure effects when they fall into the galaxy cluster potential. As a consequence, their gas is stripped to form extended tails within which star formation can happen, giving them the typical jellyfish appearance. The ultraviolet imaging observations of jellyfish galaxies provide an opportunity to understand ongoing star formation in the stripped tails. We report the ultraviolet observations of the jellyfish galaxies JW39, JO60, JO194 and compare with observations in optical continuum and $\mathrm{H}{\alpha}$. We detect knots of star formation in the disk and tails of the galaxies and find that their UV and H$\alpha$ flux are well correlated. The optical emission line ratio maps of these galaxies are used to identify for every region the emission mechanism, due to either star formation, LINER or a mix of the two phenomena. The star-forming regions in the emission line maps match very well with the regions having significant UV flux. The central regions of two galaxies (JW39, JO194) show a reduction in UV flux which coincides with composite or LINER regions in the emission line maps. The galaxies studied here demonstrate significant star formation in the stripped tails, suppressed star formation in the central regions and present a possible case of accelerated quenching happening in jellyfish galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02458-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02458) | **Methods for quasar absorption system measurements of the fine structure  constant in the 2020s and beyond**  |
|| Dinko Milaković, <mark>Chung-Chi Lee</mark>, Paolo Molaro, John K. Webb |
|*Appeared on*| *2022-12-06*|
|*Comments*| *10 pages, part of the HACK100 conference proceedings*|
|**Abstract**| This article reviews the two major recent developments that significantly improved cosmological measurements of fundamental constants derived from high resolution quasar spectroscopy. The first one is the deployment of astronomical Laser Frequency Combs on high resolution spectrographs and the second one is the development of spectral analysis tools based on Artificial Intelligence methods. The former all but eliminated the previously dominant source of instrumental uncertainty whereas the latter established optimal methods for measuring the fine structure constant ($\alpha$) in quasar absorption systems. The methods can be used on data collected by the new ESPRESSO spectrograph and the future ANDES spectrograph on the Extremely Large Telescope to produce unbiased $\Delta\alpha/\alpha$ measurements with unprecedented precision. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.02491-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.02491) | **Evolution of the Mass-Metallicity Relation from Redshift $z\approx8$ to  the Local Universe**  |
|| <mark>Danial Langeroodi</mark>, et al. -- incl., <mark>Yu-Heng Lin</mark> |
|*Appeared on*| *2022-12-06*|
|*Comments*| *Submitted to ApJ, 29 pages, 18 figures, 3 tables*|
|**Abstract**| A tight positive correlation between the stellar mass and the gas-phase metallicity of galaxies has been observed at low redshifts, with only $\sim 0.1$ dex scatter in metallicity. The shape and normalization of this correlation can set strong constraints on theories of galaxy evolution. In particular, its redshift evolution is thought to be determined by stellar and active galactic nucleus feedback-driven outflows, the redshift evolution of the stellar initial mass function or stellar yields, and broadly the star-formation histories of galaxies. The advent of \jwst\ allows probing the mass--metallicity relation at redshifts far beyond what was previously accessible. Here we report the discovery of two emission-line galaxies at redshift $z = 8.15$ and $z = 8.16$ in \jwst\ NIRCam imaging and NIRSpec spectroscopy of galaxies gravitationally lensed by the cluster RX\,J2129.4$+$0009. We measure their metallicities using the strong-line method and their stellar masses through spectral-energy-distribution fitting with a nonparametric star-formation history. We combine these with nine similarly re-analysed galaxies at $7.2 < z < 9.5$ to compile a sample of eleven galaxies at $z \approx 8$ (six with \jwst\ metallicities and five with ALMA metallicities). Based on this sample, we report the first quantitative statistical inference of the mass--metallicity relation at $z\approx8$ (median $z = 8.15$). We measure a $\sim 1.0$ dex redshift evolution in the normalization of the mass--metallicity relation from $z \approx 8$ to the local Universe; at fixed stellar mass, galaxies are 10 times less metal enriched at $z \approx 8$ compared to the present day (abridged). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.01493-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.01493) | **Applications of AI in Astronomy**  |
|| S. G. Djorgovski, et al. -- incl., <mark>A. A. Mahabal</mark> |
|*Appeared on*| *2022-12-06*|
|*Comments*| *12 pages, 1 figure, an invited review chapter, to appear in: Artificial Intelligence for Science, eds. A. Choudhary, G. Fox and T. Hey, Singapore: World Scientific, in press (2023)*|
|**Abstract**| We provide a brief, and inevitably incomplete overview of the use of Machine Learning (ML) and other AI methods in astronomy, astrophysics, and cosmology. Astronomy entered the big data era with the first digital sky surveys in the early 1990s and the resulting Terascale data sets, which required automating of many data processing and analysis tasks, for example the star-galaxy separation, with billions of feature vectors in hundreds of dimensions. The exponential data growth continued, with the rise of synoptic sky surveys and the Time Domain Astronomy, with the resulting Petascale data streams and the need for a real-time processing, classification, and decision making. A broad variety of classification and clustering methods have been applied for these tasks, and this remains a very active area of research. Over the past decade we have seen an exponential growth of the astronomical literature involving a variety of ML/AI applications of an ever increasing complexity and sophistication. ML and AI are now a standard part of the astronomical toolkit. As the data complexity continues to increase, we anticipate further advances leading towards a collaborative human-AI discovery. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.01397.md
    + _build/html/tmp_2212.01397/./figure23.png
    + _build/html/tmp_2212.01397/./figure10.jpg
    + _build/html/tmp_2212.01397/./figure17.jpg


exported in  _build/html/2212.01526.md
    + _build/html/tmp_2212.01526/./lumdist2.png
    + _build/html/tmp_2212.01526/./colorcolor.png
    + _build/html/tmp_2212.01526/./age_mass_color.png
exported in  _build/html/2212.01707.md
    + _build/html/tmp_2212.01707/./spec_reg.png
    + _build/html/tmp_2212.01707/./spec3c297.png
    + _build/html/tmp_2212.01707/./3c279_ne5_new.png
    + _build/html/tmp_2212.01707/./3c279_o2_new.png
exported in  _build/html/2212.02235.md
    + _build/html/tmp_2212.02235/./figures/rho_mol_vs_redshift_TOTAL_and_LF-CUT.png
    + _build/html/tmp_2212.02235/./figures/rho_mol_vs_redshift_poisson_percentage.png
    + _build/html/tmp_2212.02235/./figures/CO_LFs_4.6arcmin2_3.png
    + _build/html/tmp_2212.02235/./figures/CO_number_counts.png
    + _build/html/tmp_2212.02235/./figures/CII_number_counts.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# THE STAR FORMATION HISTORY OF THE MILKY WAY'S NUCLEAR STAR CLUSTER

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.01397-b31b1b.svg)](https://arxiv.org/abs/2212.01397)<mark>Appeared on: 2022-12-06</mark> - _37 pages, 23 figures, Accepted to ApJ_

</div>
<div id="authors">

Zhuo Chen, et al. -- incl., <mark>Anja Feldmeier-Krause</mark>

</div>
<div id="abstract">

**Abstract:** We report the first star formation history study of the Milky Way’s nuclear star cluster (NSC) that includes observational constraints from a large sample of stellar metallicity measurements. These metallicity measurements were obtained from recent surveys from Gemini and VLT of 770 late-type stars within the central 1.5 pc. These metallicity measurements, along with photometry and spectroscopically derived temperatures, are forward modeled with a Bayesian inference approach.  Including metallicity measurements improves the overall fit quality, as the low-temperature red giants that were previously difficult to constrain are now accounted for, and the best fit favors a two-component model. The dominant component contains 93\%$\pm$3\%of the mass, is metal-rich ($\overline{[M/H]}\sim$0.45), and has an age of 5$^{+3}_{-2}$Gyr, which is$\sim$3 Gyr younger than earlier studies with fixed (solar) metallicity; this younger age challenges co-evolutionary models in which the NSC and supermassive black holes formed simultaneously at early times. The minor population component has low metallicity ($\overline{[M/H]}\sim$-1.1) and contains$\sim$7\%of the stellar mass. The age of the minor component is uncertain (0.1 - 5 Gyr old). Using the estimated parameters, we infer the following NSC stellar remnant population (with$\sim$18\%uncertainty): 1.5$\times$10$^5$neutron stars, 2.5$\times$10$^5$stellar mass black holes (BHs) and 2.2$\times$10$^4$BH-BH binaries. These predictions result in 2-4 times fewer neutron stars compared to earlier predictions that assume solar metallicity, introducing a possible new path to understand the so-called "missing pulsar problem”.  Finally, we present updated predictions for the BH-BH merger rates (0.01-3 Gpc$^{-3}$yr$^{-1}$).

</div>

<div id="div_fig1">

<img src="tmp_2212.01397/./figure23.png" alt="Fig7" width="100%"/>

**Figure 7. -** Comparison of the metallicity measurements for the 27 common stars from both datasets. Blue crosses show the metallicities as measured from the AO spectra using MARCS grid \citep{Do et al. 2015}, compared to those as measured from the seeing-limited spectra using PHOENIX grid \citep{Feldmeier-Krause et al. 2017}. We investigated the effects of spectral resolution and grids on the measurements by re-fitting the AO spectra using the PHOENIX grid (see orange open circles with updated x-axis values). The grey error bar on the lower right shows average uncertainties. \label{fig:mh_grids_comp} (*fig:mh_grids_comp*)

</div>
<div id="div_fig2">

<img src="tmp_2212.01397/./figure10.jpg" alt="Fig13" width="100%"/>

**Figure 13. -** IMF and metallicity are crucial properties for predicting the number of compact objects and their merger rates at the Galactic center. \textbf{Left panel}: The high metallicity of the main population of the NSC ($\overline{[M/H]}$ = 0.35, blue bar) predicts 2 - 4 times fewer neutron stars than those assuming a solar metallicity ($\overline{[M/H]}$ = 0, yellow bar). \textbf{Middle panel}: A cluster with a top-heavy IMF ($\alpha$ = -1.7 $\pm$ 0.2, red) produces 3 - 6 times more black holes and 2 - 3 times more neutron stars than a cluster with a Kroupa IMF. \textbf{Right panel}: Comparison of the predicted BH-BH merger rate per volume assuming different IMFs, as calculated from a range of possible 3-d radial density profile of BHs with a power-law indice range of 7/4 $<$$\beta$$<$ 11/4 \citep{Alexander et al. 2009} covering both severe and weak (Buhcall-Wolf profile) mass segregation scenarios. The width in the band corresponds to the assumption of ellipticity of the NSC from 0.1 to 1. The top-heavy IMF predicts the BH-BH mergers with a rate of up to 19 times higher than that with a Kroupa IMF. \label{fig:comp_merger} (*fig:comp_merger*)

</div>
<div id="div_fig3">

<img src="tmp_2212.01397/./figure17.jpg" alt="Fig18" width="100%"/>

**Figure 18. -** \textbf{Left:} Stellar metallicity distribution from a synthetic cluster. The input cluster metallicity is shown as a vertical line. Uncertainty is added for each star as a Gaussian distribution. A random bias to $[M/H]$ is artificially introduced to simulate an apparent tail in the measured metallicity distribution out to $[M/H]$$\sim$ +1 dex. \textbf{Middle:} Test-cluster's marginalized 1D posterior probability density function for a simulated cluster with an age of 5 Gyr (log(Age) = 9.7). We find that the best fit cluster age of 5.7 $^{+3.8}_{-3.5}$ Gyr is consistent with the input age (red line). \textbf{Right:} Example for a simulated cluster with an age of 10 Gyr (log(Age) = 10). The best fit cluster age of 8.9 $^{+4.0}_{-3.6}$ Gyr is consistent with the input age (red line). Our methodology is able to recover the cluster age with no substantial systematic biases.
\label{fig:mhfittertest} (*fig:mhfittertest*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\STScI$}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$</div>



<div id="title">

#  Compact Source Population

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.01526-b31b1b.svg)](https://arxiv.org/abs/2212.01526)<mark>Appeared on: 2022-12-06</mark> - _15 pages, 5 figures, Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ_

</div>
<div id="authors">

Hamid Hassani, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>Kathryn Kreckel</mark>, <mark>Eva Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We use PHANGS-JWST data to identify and classify 1271 compact 21\umsources in four nearby galaxies using MIRI F2100W data. We identify sources using a dendrogram-based algorithm, and we measure the background-subtracted flux densities for JWST bands from 2$\um$to 21$\um$. Using the SED in JWST as well as HST bands, plus ALMA and MUSE/VLT observations, we classify the sources by eye. Then we use this classification to define regions in color-color space, and so establish a quantitative framework for classifying sources. We identify 1085 sources as belonging to the ISM of the target galaxies with the remainder being dusty stars or background galaxies.  These 21\umsources are strongly spatially associated with$\ion${H}{2}regions ($>92\%$of sources), while 74\%of sources are coincident with a stellar association defined in the HST data. Using SED fitting, we find that the stellar masses of the 21\umsources span a range of 10$^{2}$to 10$^{4} M_\odot$with mass-weighted ages down to 2 Myr. There is a tight correlation between attenuation-corrected H$\alpha$and 21\umluminosity for$L_{\nu,\mathrm{F2100W}}>10^{19} \mathrm{W Hz}^{-1}$. Young embedded source candidates selected at 21\umare found below this threshold and have$M_\star < 10^{3} M_\odot$.

</div>

<div id="div_fig1">

<img src="tmp_2212.01526/./lumdist2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Luminosity distribution of 21 \um sources classified as ISM sources in the JWST MIRI data.  The distributions are grouped by host galaxy in order of increasing distance (top to bottom). The blue dotted line indicates the mean $5\sigma$ error in source luminosity. The red lines indicate a fit to the power-law distribution with $dN/dL_\nu \propto L_\nu^{\beta}$ over two orders of magnitude. The value of $\beta$ is labeled in each panel.  The mass scale on the top axis translates the luminosity measurements to equivalent masses using the median ratio we determine from SED fitting (Section \ref{sec:sedfit}). (*fig:lumdist*)

</div>
<div id="div_fig2">

<img src="tmp_2212.01526/./colorcolor.png" alt="Fig4" width="100%"/>

**Figure 4. -** Color-color plots of the 21 \um selected sources in the MIRI (left) and NIRCam (right) bands constructed from band ratios that highlight significant PAH emission for $z=0$ sources.  Dashed lines indicate loci that partition the color-color space into sources with bright ISM emission \edit1{(ISM)} vs. background galaxies \edit1{(HZ)} and extreme AGB stars that lack a PAH feature \edit1{(STAR)}.  Classifications are made by eye in examining the full SED and morphology of sources in the full PHANGS data set. The L20 line indicates the proposed line for stellar colors from \citet{lai20}.  The green line in the right-hand plot indicates the expected flux ratios for PAH emission in \citet{SANDSTROM2_PHANGSJWST}. (*fig:colors*)

</div>
<div id="div_fig3">

<img src="tmp_2212.01526/./age_mass_color.png" alt="Fig5" width="100%"/>

**Figure 5. -** Correlation between luminosity in the extinction corrected H$\alpha$ line as a function of 21 \um luminosity density. The color bar shows age of the sources and the total stellar mass of each region is represented by its size. Both age and mass are estimated via SED fitting using $\textsc${cigale}(Section \ref{sec:sedfit}). Most regions show an excellent correlation between 21 \um and H$\alpha$ luminosity.  The black dashed line indicates the threshold below which we regard a 21 \um source as being H$\alpha$-faint \edit1{($L_\mathrm{H\alpha} / \nu L_{\nu, \mathrm{F2100W}}=0.05<1/80$)}, and the red line indicates the mean $L_\mathrm{H\alpha} / \nu L_{\nu, \mathrm{F2100W}}=0.05$.  The vertical line at $L_\nu = 10^{19} \mathrm{W Hz^{-1}}$ is the threshold below which we search for embedded sources. \edit1{We plot representative fractional error bars in the bottom of the plot in five different $L_{\nu, \mathrm{F2100W}}$ ranges.} (*fig:hacorr*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\chn}{{\it Chandra}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\chn$}{{\it Chandra}}$</div>



<div id="title">

# ?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.01707-b31b1b.svg)](https://arxiv.org/abs/2212.01707)<mark>Appeared on: 2022-12-06</mark> - _The Astrophysical Journal Supplement Series, in press_

</div>
<div id="authors">

Valentina Missaglia, et al. -- incl., <mark>Mischa Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** The environment of the high-redshift ($z=1.408$), powerful radio-loud galaxy 3C 297 has several distinctive features of a galaxy cluster. Among them, a characteristic halo of hot gas revealed by Chandra X-ray observations. In addition, a radio mapobtained with the Very Large Array (VLA) shows a bright hotspot in the northwestern direction, created by the interaction of the AGN jet arising from 3C 297 with its environment. In the X-ray images, emission cospatial with the northwestern radio lobe is detected, and peaks at the position of the radio hotspot. The extended, complex X-ray emission observed with our new Chandra data is largely unrelated to its radio structure. Despite having attributes of a galaxy cluster, no companion galaxies have been identified from 39 new spectra of neighboring targets of 3C 297 obtained with the Gemini Multi-Object Spectrograph. None of the 19 galaxies for which a redshift was determined lies at the same distance as 3C 297. The optical spectral analysis of the new Gemini spectrum of 3C 297 reveals an isolated Type-II radio-loud AGN. We also detected line broadening in[O{\sc ii}]$\lambda$3728 with a FWHM about 1700 km s$^{-1}$and possible line shifts of up to 500-600 km s$^{-1}$. We postulate that the host galaxy of 3C 297 is a fossil group, in which most of the stellar mass has merged into a single object, leaving behind an X-ray halo.

</div>

<div id="div_fig1">

<img src="tmp_2212.01707/./spec_reg.png" alt="Fig2" width="100%"/>

**Figure 2. -** Unbinned, unsmoothed $\chn$ merged image of 3C 297 in the energy band 0.3-7 keV. The three regions in which spectral analyses have been performed are highlighted in the blue regions (two circles and one ellipse). We detected 159 net counts in the extended emission region, 115 net counts in the nuclear region and 315 net counts in the hotspot region. The red dashed circle and ellipse mark the regions in which we observe an X-ray counts excess, due to the outflow (see Section \ref{outflow} for more details). \\label{regions} (*regions*)

</div>
<div id="div_fig2">

<img src="tmp_2212.01707/./spec3c297.png" alt="Fig5" width="100%"/>

**Figure 5. -** New GMOS spectrum of 3C 297. The spectrum is displayed with a redshift correction of $z$=1.408.
\label{spec3c297} (*spec3c297*)

</div>
<div id="div_fig3">

<img src="tmp_2212.01707/./3c279_ne5_new.png" alt="Fig6.1" width="50%"/><img src="tmp_2212.01707/./3c279_o2_new.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Example of the Gaussian decomposition applied to the observed emission line profiles.
\textit{Left}: Fit to the coronal lines [Ne {\sc v}] $\lambda\lambda$ 3345, 3425. \textit{Right}:
[O {\sc ii}] $\lambda\lambda$ 37286, 3729. For this latter line, a second blueshifted, broad component of
FWHM=1790 km s$^{-1}$ was necessary to reproduce the observed profile. In both panels, the
observed profile is in black, individual Gaussians are in blue, the total fit in red. The green
line is the continuum level and the dashed maroon line is the residual after subtraction of the fit.
\label{fig:gaussfit} (*fig:gaussfit*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

#  field and the impact of field-to-field variance

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.02235-b31b1b.svg)](https://arxiv.org/abs/2212.02235)<mark>Appeared on: 2022-12-06</mark> - __

</div>
<div id="authors">

A. Gkogkou, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** In the submillimeter regime, spectral line scans and line intensity mapping (LIM) are new promising probes for the cold gas content and star formation rate of galaxies across cosmic time. However, both of these two measurements suffer from field-to-field variance. We study the effect of field-to-field variance on the predicted CO and[CII]power spectra from future LIM experiments such as CONCERTO, as well as on the line luminosity functions (LFs) and the cosmic molecular gas mass density that are currently derived from spectral line scans. We combined a$\rm 117   deg^2$dark matter lightcone from the Uchuu cosmological simulation with the simulated infrared dusty extragalactic sky (SIDES) approach. The clustering of the dusty galaxies in the SIDES-Uchuu product is validated by reproducing the cosmic infrared background anisotropies measured by\textit{Herschel}and\textit{Planck}. We find that in order to constrain the CO LF with an uncertainty below 20\%, we need survey sizes of at least$\rm 0.1   deg^2$. Furthermore, accounting for the field-to-field variance using only the Poisson variance can underestimate the total variance by up to 80\%. The lower the luminosity is and the larger the survey size is, the higher the level of underestimate. At$z<3$, the impact of field-to-field variance on the cosmic molecular gas density can be as high as 40\%for the 4.6 arcmin$^2$field, but drops below 10\%for areas larger than 0.2 deg$^2$. However, at$z>3$the variance decreases more slowly with survey size and for example drops below 10\%for 1 deg$^2$fields. Finally, we find that the CO and[CII]LIM power spectra can vary by up to 50\%in$\rm 1   deg^2$fields. This limits the accuracy of the constraints provided by the first 1 deg$^2$surveys. In addition the level of the shot noise power is always dominated by the sources that are just below the detection thresholds, which limits its potential for deriving number densities of faint[CII]emitters. We provide an analytical formula to estimate the field-to-field variance of current or future LIM experiments given their observed frequency and survey size. The underlying code to derive the field-to-field variance and the full SIDES-Uchuu products (catalogs, cubes, and maps) are publicly available.

</div>

<div id="div_fig1">

<img src="tmp_2212.02235/./figures/rho_mol_vs_redshift_TOTAL_and_LF-CUT.png" alt="Fig14.1" width="50%"/><img src="tmp_2212.02235/./figures/rho_mol_vs_redshift_poisson_percentage.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** Field-to-field variance of the molecular gas mass density, $\rm \rho_{H_2}$, at several redshift slices where observational data are available. Left: Evolution of $\rm \rho_{H_2}$ with redshift. The empty boxes are the observational data from ASPECS \citep{decarli2020} and COLDz \citep{riechers2019}. We also include the upper limit offered by COLDz assuming that all the galaxies could be CO(2-1) emitters at high z. The black dashed line is the SIDES model without a luminosity cut and the black dotted line is the SIDES model with a luminosity cut at $\rm \sim 5 \times 10^{9}   K  km   s^{-1} pc^2$. The steps in both lines are caused by the selected redshift grid that the cosmological simulation snapshots were taken. The shaded gray areas correspond to the $\rm \rho_{H_2}$ field-to-field variance which has been computed for the same redshift ranges and sizes as the corresponding surveys. Right: Field-to-field variance in the molecular gas density in different redshift bins. The light-colored areas show the total field-to-field variance introduced in the molecular gas density estimates, while the dark-colored areas show the Poisson-only variance. The contribution of the latter to the total variance is given as a percentage on top of each rectangle. The different colors indicate the different surveys, hence different size and redshift slice. (*fig:rho_mol_vs_redshift*)

</div>
<div id="div_fig2">

<img src="tmp_2212.02235/./figures/CO_LFs_4.6arcmin2_3.png" alt="Fig13" width="100%"/>

**Figure 13. -** Comparison of the CO LF resulting from the SIDES simulation with ASPECS observational data. Each LF is created using sources from 117 different $\rm 4.6   arcmin^2$-sized simulated subfields. This field size is chosen to match the size of ASPECS. The gray shaded areas with different transparencies correspond to the 16th-84th and 5th-95th percentile confidence intervals. The black solid line is the LF of the entire volume of the Uchuu simulation, and the dashed line is the median of the multiple LFs computed from all the subfields. The red line shows the resulting LF presented in B22 where the Bolshoi-Planck cosmological simulation was used, while the arrows show the last luminosity bin of the SIDES-Uchuu LFs that contains at least one source. The different colors stand for a different number of sources. (*fig:CO_LF*)

</div>
<div id="div_fig3">

<img src="tmp_2212.02235/./figures/CO_number_counts.png" alt="Fig8.1" width="50%"/><img src="tmp_2212.02235/./figures/CII_number_counts.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Source flux contribution to the CO (top) and [CII](bottom) shot noise power spectrum (see Eq. \ref{eq:shot_noise}). The different color lines correspond to different frequency slices with the CONCERTO frequency resolution ($\rm1.5   GHz$) within the whole CONCERTO observing frequency range. The dashed lines are the expected shot noise level as computed by Eq. \ref{eq:shot_noise}. (*fig:number_counts*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

125  publications in the last 7 days.
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers